In [12]:
import os
import pandas as pd
import numpy as np
import torch as t
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import scipy.io

In [13]:
data_dir = '/Users/tyler/Datasets/deepsat-sat6/'

In [14]:
os.listdir(data_dir)

['y_test_sat6.csv',
 'sat-6-full.mat',
 'X_train_sat6.csv',
 'sat6annotations.csv',
 'y_train_sat6.csv',
 'X_test_sat6.csv']

In [5]:
class SatelliteDataset(Dataset):
    def __init__(self, X_csv, y_csv, transform=None):
        self.instances = pd.read_csv(X_csv)
        self.labels = pd.read_csv(y_csv)
        self.transform = transform
    
    def __len__(self): return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.instances.iloc[idx]
        label = self.labels.iloc[idx]
        return {'chip': img, 'label': label}

In [7]:
training_set = SatelliteDataset(X_csv=data_dir+'X_train_sat6.csv', y_csv=data_dir+'y_train_sat6.csv')

In [8]:
validation_set = SatelliteDataset(X_csv=data_dir+'X_test_sat6.csv', y_csv=data_dir+'y_test_sat6.csv')

KeyboardInterrupt: 